In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Configuration
IMG_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 50

In [5]:
import os
import pickle
from datetime import datetime

def save_processed_data(images, labels, label_map, save_dir='processed_data'):
    """
    Save the processed images, labels, and label map to disk

    Args:
        images (np.array): Processed face images
        labels (np.array): Corresponding labels
        label_map (dict): Mapping of label indices to person names
        save_dir (str): Directory to save the processed data
    """
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Generate timestamp for the filename
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Save the data
    data = {
        'images': images,
        'labels': labels,
        'label_map': label_map,
        'timestamp': timestamp
    }

    filepath = os.path.join(save_dir, f'processed_data_{timestamp}.pkl')
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)

    print(f"Data saved to: {filepath}")

    # Save metadata for easy loading
    metadata = {
        'latest_file': filepath,
        'n_images': len(images),
        'n_classes': len(label_map)
    }

    with open(os.path.join(save_dir, 'metadata.pkl'), 'wb') as f:
        pickle.dump(metadata, f)

def load_processed_data(filepath=None, data_dir='processed_data'):
    """
    Load previously processed data from disk

    Args:
        filepath (str, optional): Specific file to load. If None, loads the latest file
        data_dir (str): Directory containing the processed data

    Returns:
        tuple: (images, labels, label_map)
    """
    if filepath is None:
        # Load metadata to get the latest file
        metadata_path = os.path.join(data_dir, 'metadata.pkl')
        if not os.path.exists(metadata_path):
            raise FileNotFoundError("No metadata file found. Please process the data first.")

        with open(metadata_path, 'rb') as f:
            metadata = pickle.load(f)
        filepath = metadata['latest_file']

    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Data file not found: {filepath}")

    print(f"Loading data from: {filepath}")
    with open(filepath, 'rb') as f:
        data = pickle.load(f)

    return data['images'], data['labels'], data['label_map']

def load_and_preprocess_data(save_dir='processed_data', force_reprocess=False):
    """
    Load and preprocess images from the dataset using kagglehub with caching

    Args:
        save_dir (str): Directory to save/load processed data
        force_reprocess (bool): If True, reprocess data even if cached version exists

    Returns:
        tuple: (images, labels, label_map)
    """
    # Check for existing processed data
    if not force_reprocess and os.path.exists(save_dir):
        try:
            return load_processed_data(data_dir=save_dir)
        except Exception as e:
            print(f"Error loading cached data: {e}")
            print("Proceeding with data processing...")

    # Download dataset
    print("Downloading dataset...")
    dataset_path = kagglehub.dataset_download("hereisburak/pins-face-recognition")
    print("Dataset downloaded to:", dataset_path)

    # Find the main dataset directory
    main_dir = None
    for item in os.listdir(dataset_path):
        if "105_classes_pins_dataset" in item:
            main_dir = os.path.join(dataset_path, item)
            break

    if main_dir is None:
        raise Exception("Could not find the main dataset directory")

    # Initialize lists to store images and labels
    images = []
    labels = []
    label_map = {}
    current_label = 0

    # Load face detection model
    face_net = cv2.dnn.readNetFromCaffe(
        'deploy.prototxt',
        'res10_300x300_ssd_iter_140000.caffemodel'
    )

    # Walk through the person directories (prefixed with "pins_")
    for person_dir in sorted(os.listdir(main_dir)):
        if person_dir.startswith('pins_'):
            person_name = person_dir[5:]  # Remove 'pins_' prefix
            person_path = os.path.join(main_dir, person_dir)

            print(f"Processing images for: {person_name}")
            label_map[current_label] = person_name

            for img_name in os.listdir(person_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(person_path, img_name)

                    # Read and preprocess image
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"Warning: Could not read image {img_path}")
                        continue

                    # Face detection
                    (h, w) = img.shape[:2]
                    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0,
                                               (300, 300), (104.0, 177.0, 123.0))
                    face_net.setInput(blob)
                    detections = face_net.forward()

                    # Get the face with highest confidence
                    max_conf = 0
                    max_face = None

                    for i in range(0, detections.shape[2]):
                        confidence = detections[0, 0, i, 2]
                        if confidence > 0.5:  # Minimum confidence threshold
                            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                            (startX, startY, endX, endY) = box.astype("int")

                            # Ensure coordinates are within image bounds
                            startX = max(0, startX)
                            startY = max(0, startY)
                            endX = min(w, endX)
                            endY = min(h, endY)

                            # Extract and preprocess face
                            face = img[startY:endY, startX:endX]
                            if face.size == 0:
                                continue

                            face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
                            if confidence > max_conf:
                                max_conf = confidence
                                max_face = face

                    if max_face is not None:
                        # Normalize pixel values
                        max_face = max_face.astype('float32') / 255.0
                        images.append(max_face)
                        labels.append(current_label)

            print(f"Processed {len([l for l in labels if l == current_label])} images for {person_name}")
            current_label += 1

    if len(images) == 0:
        raise Exception("No images were successfully processed")

    # Convert to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    # Save the processed data
    save_processed_data(images, labels, label_map, save_dir)

    return images, labels, label_map

In [6]:
# Load and preprocess data
print("Loading and preprocessing data...")
X, y, label_map = load_and_preprocess_data()

Loading and preprocessing data...
Dataset downloaded to: /root/.cache/kagglehub/datasets/hereisburak/pins-face-recognition/versions/1
Processing images for: Adriana Lima
Processed 213 images for Adriana Lima
Processing images for: Alex Lawther
Processed 152 images for Alex Lawther
Processing images for: Alexandra Daddario
Processed 225 images for Alexandra Daddario
Processing images for: Alvaro Morte
Processed 139 images for Alvaro Morte
Processing images for: Amanda Crew
Processed 117 images for Amanda Crew
Processing images for: Andy Samberg
Processed 196 images for Andy Samberg
Processing images for: Anne Hathaway
Processed 203 images for Anne Hathaway
Processing images for: Anthony Mackie
Processed 124 images for Anthony Mackie
Processing images for: Avril Lavigne
Processed 162 images for Avril Lavigne
Processing images for: Ben Affleck
Processed 126 images for Ben Affleck
Processing images for: Bill Gates
Processed 122 images for Bill Gates
Processing images for: Bobby Morley
Proc

In [7]:
def create_model(input_shape, num_classes):
    """
    Create an enhanced ANN model architecture with advanced techniques for better accuracy
    """
    model = Sequential([
        # Input preprocessing
        Flatten(input_shape=input_shape),
        BatchNormalization(),

        # First dense block
        Dense(2048, kernel_initializer='he_uniform'),
        BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.1),
        Dropout(0.4),

        # Second dense block with residual connection
        Dense(1024, kernel_initializer='he_uniform'),
        BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.1),
        Dropout(0.4),

        # Third dense block
        Dense(512, kernel_initializer='he_uniform'),
        BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.1),
        Dropout(0.3),

        # Feature extraction block
        Dense(256, kernel_initializer='he_uniform'),
        BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.1),
        Dropout(0.3),

        # Deep features block
        Dense(128, kernel_initializer='he_uniform'),
        BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.1),
        Dropout(0.2),

        # Classification head
        Dense(num_classes, activation='softmax',
              kernel_initializer='glorot_uniform',
              kernel_regularizer=tf.keras.regularizers.l2(0.01))
    ])

    # Custom learning rate schedule
    initial_learning_rate = 0.001
    decay_steps = 1000
    decay_rate = 0.9
    learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps, decay_rate
    )

    # Custom optimizer with parameters
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=learning_rate_schedule,
        weight_decay=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07
    )

    # Compile with custom settings
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            'accuracy',
            tf.keras.metrics.SparseCategoricalAccuracy(name='categorical_accuracy'),
            tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy')
        ]
    )

    return model

def train_model(model, X_train, y_train, X_test, y_test, epochs=50, batch_size=32):
    """
    Enhanced training function with advanced callbacks and monitoring
    """
    # Define initial_learning_rate here
    initial_learning_rate = 0.001

    # Enhanced data augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.1,
        fill_mode='nearest',
        brightness_range=[0.8, 1.2],
        preprocessing_function=tf.keras.applications.imagenet_utils.preprocess_input
    )

    # Advanced callbacks
    callbacks = [
        # Early stopping with restoration
        EarlyStopping(
            monitor='val_categorical_accuracy',
            patience=15,
            restore_best_weights=True,
            mode='max'
        ),

        # Adaptive learning rate reduction
        ReduceLROnPlateau(
            monitor='val_categorical_accuracy',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            mode='max',
            verbose=1
        ),

        # Model checkpoint
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.keras',  # Change the filepath to end with .keras
            monitor='val_categorical_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]

 # Train with advanced settings
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        validation_data=(X_test, y_test),
        epochs=epochs,
        callbacks=callbacks,
        # Removed 'workers' and 'use_multiprocessing' arguments
        shuffle=True
    )

    return history, model

def plot_training_history(history):
    """
    Plot training and validation metrics
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(['Train', 'Validation'])

    # Plot loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(['Train', 'Validation'])

    plt.tight_layout()
    plt.show()


In [9]:
# Load the preprocessed data
X, y, label_map = load_processed_data()

print(f"Dataset summary:")
print(f"Total number of images: {len(X)}")
print(f"Number of classes: {len(label_map)}")
print(f"Images per class:")
for label, name in label_map.items():
    count = np.sum(y == label)
    print(f"{name}: {count} images")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# Create data generator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create and train model with enhanced settings
model = create_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=len(label_map))

# Print model summary
model.summary()

# Train with enhanced function
history, trained_model = train_model(
    model, X_train, y_train, X_test, y_test,
    epochs=9,
    batch_size=32
)

# Evaluate model
print("\nEvaluating model...")
test_loss, test_accuracy, cat_accuracy, top_5_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy*100:.2f}%")
print(f"Top-5 accuracy: {top_5_accuracy*100:.2f}%")


# Generate predictions and classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes,
                          target_names=[label_map[i] for i in range(len(label_map))]))

# Plot training history
plot_training_history(history)

# Plot confusion matrix
plt.figure(figsize=(12, 8))
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=list(label_map.values()), yticklabels=list(label_map.values()))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


Loading data from: processed_data/processed_data_20241027_164743.pkl
Dataset summary:
Total number of images: 17533
Number of classes: 105
Images per class:
Adriana Lima: 213 images
Alex Lawther: 152 images
Alexandra Daddario: 225 images
Alvaro Morte: 139 images
Amanda Crew: 117 images
Andy Samberg: 196 images
Anne Hathaway: 203 images
Anthony Mackie: 124 images
Avril Lavigne: 162 images
Ben Affleck: 126 images
Bill Gates: 122 images
Bobby Morley: 138 images
Brenton Thwaites: 209 images
Brian J. Smith: 102 images
Brie Larson: 169 images
Chris Evans: 166 images
Chris Hemsworth: 159 images
Chris Pratt: 176 images
Christian Bale: 154 images
Cristiano Ronaldo: 98 images
Danielle Panabaker: 181 images
Dominic Purcell: 146 images
Dwayne Johnson: 141 images
Eliza Taylor: 162 images
Elizabeth Lail: 158 images
Emilia Clarke: 210 images
Emma Stone: 139 images
Emma Watson: 211 images
Gwyneth Paltrow: 187 images
Henry Cavil: 195 images
Hugh Jackman: 179 images
Inbar Lavi: 127 images
Irina Shayk: 1

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 30000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 30000)               │         120,000 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 2048)                │      61,442,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_6 (LeakyReLU)            │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_7 (LeakyReLU)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_8 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │          32,8

 Total params: 64,378,665 (245.59 MB)

...

 Non-trainable params: 67,936 (265.38 KB)

Epoch 1/9


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


437/439 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.0101 - categorical_accuracy: 0.0101 - loss: 5.5176 - top_5_accuracy: 0.0534
Epoch 1: val_categorical_accuracy improved from -inf to 0.00656, saving model to best_model.keras
439/439 ━━━━━━━━━━━━━━━━━━━━ 362s 801ms/step - accuracy: 0.0101 - categorical_accuracy: 0.0101 - loss: 5.5155 - top_5_accuracy: 0.0534 - val_accuracy: 0.0066 - val_categorical_accuracy: 0.0066 - val_loss: 19188.3828 - val_top_5_accuracy: 1.0000 - learning_rate: 9.5480e-04
Epoch 2/9
438/439 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.0109 - categorical_accuracy: 0.0109 - loss: 4.7849 - top_5_accuracy: 0.0548
Epoch 2: val_categorical_accuracy improved from 0.00656 to 0.00912, saving model to best_model.keras
439/439 ━━━━━━━━━━━━━━━━━━━━ 358s 816ms/step - accuracy: 0.0109 - categorical_accuracy: 0.0109 - loss: 4.7847 - top_5_accuracy: 0.0548 - val_accuracy: 0.0091 - val_categorical_accuracy: 0.0091 - val_loss: 32994.5898 - val_top_5_accuracy: 1.0000 - learn

TypeError: This optimizer was created with a `LearningRateSchedule` object as its `learning_rate` constructor argument, hence its learning rate is not settable. If you need the learning rate to be settable, you should instantiate the optimizer with a float `learning_rate` argument.